In [1]:
import os
import numpy as np
import pandas as pd
os.chdir(path=os.getcwd().rstrip('\\code'))

# Get data

#### IPO data

In [2]:
ipo = pd.read_excel(io='data/ipo_raw.xlsx', sheet_name=0)
ipo = ipo.set_index(keys='date')
ipo = ipo.sort_index(ascending=False)
ipo

,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price
date,,,,,,
2022-11-10,TENDERHUT*,44,NaN,NaN,41.00,NaN
2022-09-13,CASPAR ASSET MANAGEMENT S.A.*,20,NaN,NaN,18.50,NaN
2022-09-05,BIOCELTIX S.A.*,52.8,NaN,NaN,52.00,NaN
2022-08-12,SIMFABRIC*,14.9,NaN,NaN,15.10,NaN
2022-07-11,CREOTECH INSTRUMENTS*,"146,00/100,00",39655800,39655800,142.00,128.8
...,...,...,...,...,...,...
2005-02-10,Ciech,24,196895616,277317024,28.10,28.0
2005-02-04,Eurocash,3.1,NaN,178200090,3.26,NaN
2005-01-27,Zelmer,13.2,NaN,169038000,17.50,NaN


In [3]:
ipo.dtypes

company_name                            object
issue_or_reference_price                object
new_issue_value                         object
new_issue_and_existing_shares_value     object
shares_close_price                     float64
pda_close_price                        float64
dtype: object

#### Stock quotes data

In [4]:
stocks = pd.read_pickle(filepath_or_buffer='data/stock_quotes_1.pkl')
stocks = stocks[['company_name', 'ISIN']].copy()
stocks

,company_name,ISIN
date,,
2005-01-03,01NFI,PLNFI0100011
2005-01-03,04PRO,PLNFI0400015
2005-01-03,05VICT,PLNFI0500012
2005-01-03,06MAGNA,PLNFI0600010
2005-01-03,08OCTAVA,PLNFI0800016
...,...,...
2022-11-15,ZEPAK,PLZEPAK00012
2022-11-15,ZPUE,PLZPUE000012
2022-11-15,ZREMB,PLZBMZC00019


# Data preprocessing

#### Preprocess the `new_issue_and_existing_shares_value` column

In [5]:
ipo['new_issue_and_existing_shares_value'] = ipo['new_issue_and_existing_shares_value'].astype(dtype=float)

#### Preprocess the `new_issue_value` column

In [6]:
ipo['new_issue_value'] = ipo['new_issue_value'].astype(dtype=float)

#### Add a new column `existing_shares_value`

In [7]:
# Case 1: companies with both *new_issue_value* and *new_issue_and_existing_shares_value*
boolean_mask = (~ipo['new_issue_value'].isna()) & (~ipo['new_issue_and_existing_shares_value'].isna())
print(f'''Case 1: There are {ipo.loc[boolean_mask].shape[0]} IPOs with both *new_issue_value* and *new_issue_and_existing_shares_value*
        meaning that companies offered both existing and new shares.''')

# Add a new column
ipo.loc[boolean_mask, 'existing_shares_value'] = \
    ipo.loc[boolean_mask, 'new_issue_and_existing_shares_value'] - ipo.loc[boolean_mask, 'new_issue_value']

# If *new_issue_value* = *new_issue_and_existing_shares_value*, set *existing_shares_value* to NaN
ipo['existing_shares_value'] = ipo['existing_shares_value'].replace(to_replace=0, value=np.nan)

Case 1: There are 307 IPOs with both *new_issue_value* and *new_issue_and_existing_shares_value*
        meaning that companies offered both existing and new shares.


In [8]:
# Case 2: companies with only *new_issue_and_existing_shares_value*
boolean_mask = (ipo['new_issue_value'].isna()) & (~ipo['new_issue_and_existing_shares_value'].isna())
print(f'''Case 2: There are {ipo.loc[boolean_mask].shape[0]} IPOs with only *new_issue_and_existing_shares_value*
        meaning that companies offered only existing shares.''')

ipo.loc[boolean_mask, 'existing_shares_value'] = ipo.loc[boolean_mask, 'new_issue_and_existing_shares_value']

Case 2: There are 43 IPOs with only *new_issue_and_existing_shares_value*
        meaning that companies offered only existing shares.


In [9]:
print(f'''Eventually there are {ipo[~ipo['existing_shares_value'].isna()].shape[0]} companies which offered the existing shares.''')

Eventually there are 163 companies which offered the existing shares.


#### Add two boolean columns `is_offering_new_shares` and `is_offering_existing_shares`

In [10]:
ipo['is_offering_new_shares'] = ipo['new_issue_value'] > 0
ipo['is_offering_existing_shares'] = ipo['existing_shares_value'] > 0

#### Get rid of companies which didn't offer neither existing nor new shares

In [11]:
boolean_mask = ipo['is_offering_new_shares'] | ipo['is_offering_existing_shares']
ipo = ipo.loc[boolean_mask].copy()
ipo.shape

(351, 9)

### Preprocess the `company_name` column

In [12]:
ipo['company_name'] = ipo['company_name'].str.replace(pat='S.A.', repl='', regex=False)
ipo['company_name'] = ipo['company_name'].str.upper()

In [13]:
# Print how many special symbols are there
print(f'''There are {len(ipo[ipo['company_name'].str.contains(pat='*', regex=False)])} companies with single *''')
print(f'''There are {len(ipo[ipo['company_name'].str.contains(pat='^', regex=False)])} companies with single ^''')

There are 25 companies with single *
There are 2 companies with single ^


#### Add a new column `is_moving` if the company moved to WSE from either the NewConnect (`*` symbol) or CeTO (`^` symbol)

In [14]:
ipo['is_moving'] = ipo['company_name'].str.contains(pat='[*^]', regex=True)

In [15]:
# Get rid of the `*`` and `^` symbols in the company_name column
ipo['company_name'] = ipo['company_name'].str.replace(pat='[*^]', repl='', regex=True)

### Preprocess the `issue_or_reference_price` column

In [16]:
ipo['issue_or_reference_price'] = ipo['issue_or_reference_price'].astype(dtype=str)

In [17]:
# For these companies moving from the NC, this is always the reference price, not the emission price
boolean_mask = (ipo['is_moving'] == True) & (ipo['is_offering_new_shares'] == True)
ipo.loc[boolean_mask]

,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving
date,,,,,,,,,,
2022-07-11,CREOTECH INSTRUMENTS,"146,00/100,00",39655800.0,39655800.0,142.00,128.80,NaN,True,False,True
2021-12-20,BIO PLANET,"26,80/28,00",5600000.0,9824612.0,26.60,27.00,4224612.0,True,True,True
2020-12-30,MEDINICE,18,12200000.0,12200000.0,17.30,NaN,NaN,True,False,True
2020-12-14,PURE BIOLOGICS,112,54000000.0,54000000.0,99.80,94.00,NaN,True,False,True
2019-02-28,DATAWALK,"20,70/22,00",10066056.0,10066056.0,20.50,19.80,NaN,True,False,True
2018-08-29,T-BULL,"44,40/50,00",1894450.0,1894450.0,44.00,43.40,NaN,True,False,True
2017-12-28,TOWER INVESTMENTS,48.05,22500000.0,22500000.0,50.33,46.50,NaN,True,False,True
2017-11-06,VENTURE INC,4.5,30000000.0,30000000.0,5.00,NaN,NaN,True,False,True
2017-07-05,MORIZON,"2,44/1,50",5042043.0,5042043.0,1.75,1.47,NaN,True,False,True


In [18]:
# Print how many inputs with a '/' are there
boolean_mask = ipo['issue_or_reference_price'].str.contains(pat='/', regex=False)
print(f'There are {len(ipo.loc[boolean_mask])} companies with both the emission price and the reference price')

ipo.loc[boolean_mask]

There are 6 companies with both the emission price and the reference price


,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving
date,,,,,,,,,,
2022-07-11,CREOTECH INSTRUMENTS,"146,00/100,00",39655800.0,3.965580e+07,142.00,128.80,NaN,True,False,True
2021-12-20,BIO PLANET,"26,80/28,00",5600000.0,9.824612e+06,26.60,27.00,4.224612e+06,True,True,True
2019-02-28,DATAWALK,"20,70/22,00",10066056.0,1.006606e+07,20.50,19.80,NaN,True,False,True
2018-08-29,T-BULL,"44,40/50,00",1894450.0,1.894450e+06,44.00,43.40,NaN,True,False,True
2017-07-05,MORIZON,"2,44/1,50",5042043.0,5.042043e+06,1.75,1.47,NaN,True,False,True
2010-11-09,GIEŁDA PAPIERÓW WARTOŚCIOWYCH,"43,00/46.00",NaN,1.208073e+09,54.00,NaN,1.208073e+09,False,True,False


#### Correct the prices manually

In [19]:
# The emission price is 100, 146 is the closing price on NC https://www.gpw.pl/aktualnosci?ph_main_01_start=show&ph_main_01_cmn_id=112761&title=Debiut+sp%C3%B3%C5%82ki+CREOTECH+INSTRUMENTS
ipo.loc[ipo['company_name'] == 'CREOTECH INSTRUMENTS', 'issue_or_reference_price'] = '100'

# The emission price is 28, 26,80 is the closing price on NC https://www.gpw.pl/aktualnosci?ph_main_01_start=show&ph_main_01_cmn_id=111948&title=Debiut+sp%C3%B3%C5%82ki+BIO+PLANET
ipo.loc[ipo['company_name'] == 'BIO PLANET', 'issue_or_reference_price'] = '28'

# The emission price is 22, 20,70 is the closing price on NC https://www.bankier.pl/wiadomosc/Debiut-DataWalk-na-rynku-podstawowym-GPW-28-lutego-7644375.html
ipo.loc[ipo['company_name'] == 'DATAWALK', 'issue_or_reference_price'] = '22'

# The emission price is 50, 44,40 is the closing price on NC https://www.money.pl/gielda/ipo/wiadomosci/artykul/gpw-debiut-t-bull-shares-newconnect-producent,105,0,2414441.html
ipo.loc[ipo['company_name'] == 'T-BULL', 'issue_or_reference_price'] = '50'

# The emission price is 1,50, 2,44 is the closing price on NC https://www.money.pl/gielda/ipo/wiadomosci/artykul/gpw-debiut-t-bull-shares-newconnect-producent,105,0,2414441.html
ipo.loc[ipo['company_name'] == 'MORIZON', 'issue_or_reference_price'] = '1.5'

# 46 is for institional investors, 43 for individual ones https://www.money.pl/gielda/wiadomosci/artykul/shares;gpw;10;proc;zysku;na;debiucie;dzieki;instytucjom,117,0,703349.html
ipo.loc[ipo['company_name'] == 'GIEŁDA PAPIERÓW WARTOŚCIOWYCH', 'issue_or_reference_price'] = '43'

In [20]:
ipo['issue_or_reference_price'] = ipo['issue_or_reference_price'].str.replace(pat=',', repl='.', regex=False)
ipo['issue_or_reference_price'] = ipo['issue_or_reference_price'].str.replace(pat='*', repl='', regex=False)
ipo['issue_or_reference_price'] = ipo['issue_or_reference_price'].astype(dtype=float)

In [21]:
# Check if there are NaNs
boolean_mask = ipo['issue_or_reference_price'].isna()
ipo.loc[boolean_mask]

,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving
date,,,,,,,,,,
2013-12-30,GORENJE,NaN,7.045172e+07,7.045172e+07,NaN,NaN,NaN,True,False,False
2011-05-11,NOVA KREDITNA BANKA,NaN,4.144426e+08,4.144426e+08,33.6,NaN,NaN,True,False,False


In [22]:
# Correct the prices manually

# https://www.finanse.egospodarka.pl/65603,Debiut-spolki-Nova-Kreditna-na-GPW,1,48,1.html
ipo.loc[ipo['company_name'] == 'NOVA KREDITNA BANKA', 'issue_or_reference_price'] = 31.58

# https://forsal.pl/artykuly/763902,debiut-gorenje-na-gpw-cena-akcji-zostala-ustalona-na-1810-zl.html
ipo.loc[ipo['company_name'] == 'GORENJE', 'issue_or_reference_price'] = 18.10

### Add a new column `close_price`

##### Case 1: companies offering both exiting and new shares

In [23]:
# For companies offering both existing and new shares, set the *close_price* to the vwap of *shares_close*price & pda_close_price
boolean_mask = (ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (~ipo['shares_close_price'].isna()) & (~ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    (
        (ipo.loc[boolean_mask, 'shares_close_price'] * ipo.loc[boolean_mask, 'existing_shares_value']) + \
        (ipo.loc[boolean_mask, 'pda_close_price'] * ipo.loc[boolean_mask, 'new_issue_value'])
    ).div(ipo.loc[boolean_mask, 'existing_shares_value'] + ipo.loc[boolean_mask, 'new_issue_value'])
    
ipo['closing_price'] = ipo['closing_price'].round(2)

There are 59 such companies.


In [24]:
# For companies offering both existing and new shares, with only *pda_close_price*, set the *close_price* to the *pda_close_price*
boolean_mask = (ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (ipo['shares_close_price'].isna()) & (~ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    ipo.loc[boolean_mask, 'pda_close_price']

There are 29 such companies.


In [25]:
# For companies offering both existing and new shares, with only *shares_close_price*, set the *close_price* to the *shares_close_price*
boolean_mask = (ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (~ipo['shares_close_price'].isna()) & (ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    ipo.loc[boolean_mask, 'shares_close_price']

There are 31 such companies.


In [26]:
# For companies offering both existing and new shares, without *shares_close_price* & *pda_close_price* set the *close_price* MANUALLY
boolean_mask = (ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (ipo['shares_close_price'].isna()) & (ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')
ipo.loc[boolean_mask]

There are 1 such companies.


,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving,closing_price
date,,,,,,,,,,,
2010-12-20,VOTUM,3.8,7600000.0,11400000.0,NaN,NaN,3800000.0,True,True,False,NaN


In [27]:
# https://ri.votum-sa.pl/nowosci/aktualnosci/pierwszy-dzien-notowan%C2%84-votum-s-a-na-rynku-glownym-gpw/
ipo.loc[ipo['company_name'] == 'VOTUM', 'closing_price'] = '4.35'

##### Case 2: companies offering only new shares

In [28]:
# For companies offering only new shares, with both *shares_close_price* and *pda_close_price*, set the *close_price* to the *pda_close_price*
boolean_mask = (ipo['is_offering_new_shares']) & (~ipo['is_offering_existing_shares']) & (~ipo['shares_close_price'].isna()) & (~ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    ipo.loc[boolean_mask, 'pda_close_price']

There are 19 such companies.


In [29]:
# For companies offering only new shares, with only *shares_close_price*, set the *close_price* to the *shares_close_price*
boolean_mask = (ipo['is_offering_new_shares']) & (~ipo['is_offering_existing_shares']) & (~ipo['shares_close_price'].isna()) & (ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    ipo.loc[boolean_mask, 'shares_close_price']

There are 59 such companies.


In [30]:
# For companies offering only new shares, with only *pda_close_price*, set the *close_price* to the *pda_close_price*
boolean_mask = (ipo['is_offering_new_shares']) & (~ipo['is_offering_existing_shares']) & (ipo['shares_close_price'].isna()) & (~ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    ipo.loc[boolean_mask, 'pda_close_price']

There are 108 such companies.


In [31]:
# For companies offering only new shares, without *shares_close_price* & *pda_close_price*, set the *close_price* MANUALLY 
boolean_mask = (ipo['is_offering_new_shares']) & (~ipo['is_offering_existing_shares']) & (ipo['shares_close_price'].isna()) & (ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')
ipo[boolean_mask]

There are 2 such companies.


,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving,closing_price
date,,,,,,,,,,,
2013-12-30,GORENJE,18.1,70451722.05,70451722.05,NaN,NaN,NaN,True,False,False,NaN
2011-06-29,OVOSTAR UNION N.V.,62.0,93000000.00,93000000.00,NaN,NaN,NaN,True,False,False,NaN


In [32]:
# https://www.parkiet.com/gospodarka/art22104951-ovostar-union-debiut-na-minusie
ipo.loc[ipo['company_name'] == 'OVOSTAR UNION N.V.', 'closing_price'] = 61

# https://forsal.pl/artykuly/763902,debiut-gorenje-na-gpw-cena-akcji-zostala-ustalona-na-1810-zl.html
ipo.loc[ipo['company_name'] == 'GORENJE', 'closing_price'] = 16.50

##### Case 3: companies offering only existing shares

In [33]:
# For companies offering only existing shares, with both *shares_close_price* and *pda_close_price*
boolean_mask = (~ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (~ipo['shares_close_price'].isna()) & (~ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

There are 0 such companies.


In [34]:
# For companies offering only existing shares, with only *shares_close_price*, set the *close_price* to the *shares_close_price*
boolean_mask = (~ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (~ipo['shares_close_price'].isna()) & (ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    ipo.loc[boolean_mask, 'shares_close_price']

There are 41 such companies.


In [35]:
# For companies offering only existing shares, with only *pda_close_price*, set the *close_price* to the *pda_close_price*
boolean_mask = (~ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (ipo['shares_close_price'].isna()) & (~ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

ipo.loc[boolean_mask, 'closing_price'] = \
    ipo.loc[boolean_mask, 'pda_close_price']

There are 2 such companies.


In [36]:
# For companies offering only existing shares, without *shares_close_price* & *pda_close_price*, set the *close_price* manually 
boolean_mask = (~ipo['is_offering_new_shares']) & (ipo['is_offering_existing_shares']) & (ipo['shares_close_price'].isna()) & (ipo['pda_close_price'].isna())
print(f'There are {ipo[boolean_mask].shape[0]} such companies.')

There are 0 such companies.


In [37]:
ipo.dtypes

company_name                            object
issue_or_reference_price               float64
new_issue_value                        float64
new_issue_and_existing_shares_value    float64
shares_close_price                     float64
pda_close_price                        float64
existing_shares_value                  float64
is_offering_new_shares                    bool
is_offering_existing_shares               bool
is_moving                                 bool
closing_price                           object
dtype: object

In [38]:
ipo['closing_price'] = ipo['closing_price'].astype(dtype=float)

### Add a new column `ISIN`

* The ISIN code is in the stock_quotes.pkl file.  
* There is a `company_name` column, but these names are sometimes very different from the `company_name` column in the ipo dataframe.  
* The thing is that the names in the stock_quotes.pkl file are shorter, so the idea is to create a new column with a short name in the ipo dataframe, and then use it to merge the ISIN codes.

In [39]:
# Create a new column by using the first string in the `company_name` column
ipo['company_name_short'] = ipo['company_name'].str.split(pat=' ', expand=True)[0]

In [40]:
boolean_mask = ipo.duplicated(subset=['company_name_short'], keep=False)
ipo.loc[boolean_mask].sort_values(by=['company_name_short'])
print(f'''Number of all companies: {ipo.shape[0]}''')
print(f'''Number of duplicates in the *company_name_short* column: {ipo.loc[boolean_mask].sort_values(by=['company_name_short']).shape[0]}''')

Number of all companies: 351
Number of duplicates in the *company_name_short* column: 17


#### Case 1: Companies with a unique `company_name_short`

In [41]:
dictionary = dict()
shares_names = stocks['company_name'].drop_duplicates()

# For each company_name_short get the list with all the names that start with the company_name_short
for company_name_short in ipo.loc[~boolean_mask]['company_name_short']:
    dictionary[company_name_short] = list(shares_names[shares_names.str.startswith(pat=company_name_short)].unique())

matched = pd.DataFrame.from_dict(data=dictionary, orient='index')
matched.index.name = 'company_name_short'
matched.columns = ['shares_name_' + str(col) for col in matched.columns]
matched = matched.reset_index(drop=False)
print(f'Matched dataframe shape: {len(matched)}')
display(matched)

Matched dataframe shape: 334


,company_name_short,shares_name_0,shares_name_1,shares_name_2,shares_name_3,shares_name_4,shares_name_5
0,CREOTECH,CREOTECH,None,None,None,None,None
1,BIO,BIOTON,BIOMEDLUB,BIOPLANET,BIOMAXIMA,BIOCELTIX,None
2,STS,STSHOLDING,None,None,None,None,None
3,BIG,BIGCHEESE,None,None,None,None,None
4,POLTREG,POLTREG,None,None,None,None,None
...,...,...,...,...,...,...,...
329,CIECH,CIECH,None,None,None,None,None
330,EUROCASH,EUROCASH,None,None,None,None,None
331,ZELMER,ZELMER,None,None,None,None,None
332,COMP,COMP,COMPLEX,COMPERIA,COMPREMUM,None,None


##### Case 1a 
company_name_short == shares_name_0  
in this case there's no need to map the `company_name_short`

In [42]:
boolean_mask = matched['company_name_short'] == matched['shares_name_0']
matched.loc[boolean_mask]

,company_name_short,shares_name_0,shares_name_1,shares_name_2,shares_name_3,shares_name_4,shares_name_5
0,CREOTECH,CREOTECH,None,None,None,None,None
4,POLTREG,POLTREG,None,None,None,None,None
5,CAVATINA,CAVATINA,None,None,None,None,None
6,ONDE,ONDE,None,None,None,None,None
7,SHOPER,SHOPER,None,None,None,None,None
...,...,...,...,...,...,...,...
328,GRAAL,GRAAL,None,None,None,None,None
329,CIECH,CIECH,None,None,None,None,None
330,EUROCASH,EUROCASH,None,None,None,None,None
331,ZELMER,ZELMER,None,None,None,None,None


In [43]:
# Let's inspect the companies with multiple names
pd.merge(
    left=ipo.reset_index()[['date', 'closing_price', 'company_name', 'company_name_short']],
    right=matched.loc[boolean_mask][['company_name_short', 'shares_name_0', 'shares_name_1', 'shares_name_2', 'shares_name_3']],
    left_on='company_name_short',
    right_on='company_name_short',
    how='inner'
).dropna(subset=['shares_name_1'])

,date,closing_price,company_name,company_name_short,shares_name_0,shares_name_1,shares_name_2,shares_name_3
56,2012-05-28,2.17,ATM SYSTEMY INFORMATYCZNE,ATM,ATM,ATMGRUPA,ATMSI,None
174,2005-01-14,43.00,COMP,COMP,COMP,COMPLEX,COMPERIA,COMPREMUM


In [44]:
# Wrong - *ATM* in the companies dataframe should be *ATMSI*
stocks[stocks['company_name'].str.startswith(pat='ATM')].drop_duplicates(subset=['company_name'], keep='first')

,company_name,ISIN
date,,
2005-01-03,ATM,PLATMSA00013
2005-01-03,ATMGRUPA,PLATM0000021
2012-05-28,ATMSI,PLATMSI00016


In [45]:
# COMP is fine
stocks[stocks['company_name'].str.startswith(pat='COMP')].drop_duplicates(subset=['company_name'], keep='first')

,company_name,ISIN
date,,
2005-01-14,COMP,PLCMP0000017
2007-12-13,COMPLEX,PLCMPLX00014
2014-03-25,COMPERIA,PLCOMPR00010
2022-01-24,COMPREMUM,PLPZBDT00013


In [46]:
# Replace *ATM* with *ATMSI* manually
ipo.loc[ipo['company_name_short'] == 'ATM', 'company_name_short'] = 'ATMSI'

##### Case 1b
company_name_short != shares_name_0 & shares_name_1 == NaN

In [47]:
boolean_mask = (matched['company_name_short'] != matched['shares_name_0']) & ~(matched['shares_name_0'].isna()) & (matched['shares_name_1'].isna())
matched.loc[boolean_mask]

,company_name_short,shares_name_0,shares_name_1,shares_name_2,shares_name_3,shares_name_4,shares_name_5
2,STS,STSHOLDING,None,None,None,None,None
3,BIG,BIGCHEESE,None,None,None,None,None
10,CAPTOR,CAPTORTX,None,None,None,None,None
15,PCF,PCFGROUP,None,None,None,None,None
23,SILVAIR,SILVAIR-REGS,None,None,None,None,None
25,TEN,TENDERHUT,None,None,None,None,None
28,TOWER,TOWERINVT,None,None,None,None,None
30,VENTURE,VENTUREIN,None,None,None,None,None
35,DINO,DINOPL,None,None,None,None,None
45,I2,I2DEV,None,None,None,None,None


In [48]:
# Replace the *company_name_short* with the accurate names
ipo['company_name_short'] = ipo['company_name_short'].str.replace(pat=u'\xa0', repl=' ').str.rstrip().replace(
    to_replace=matched.loc[boolean_mask].set_index(keys='company_name_short')['shares_name_0']
)

##### Case 1c
company_name_short != shares_name_0 & shares_name_0 == NaN

In [49]:
boolean_mask = (matched['company_name_short'] != matched['shares_name_0']) & (matched['shares_name_0'].isna())
matched.loc[boolean_mask]

,company_name_short,shares_name_0,shares_name_1,shares_name_2,shares_name_3,shares_name_4,shares_name_5
11,"HUUUGE,",None,None,None,None,None,None
12,ANSWEAR.COM,None,None,None,None,None,None
17,ALLEGRO.EU,None,None,None,None,None,None
18,GAMING,None,None,None,None,None,None
19,GAMES,None,None,None,None,None,None
...,...,...,...,...,...,...,...
306,PUŁAWY,None,None,None,None,None,None
315,TRAVELPLANET.PL,None,None,None,None,None,None
322,POLISH,None,None,None,None,None,None
326,ZĄBKOWICE-ERG,None,None,None,None,None,None


In [50]:
# Create a dictionary manually
dictionary = {
    'HUUUGE,': 'HUUUGE',
    'ANSWEAR.COM': 'ANSWEAR',
    'ALLEGRO.EU': 'ALLEGRO',
    'GAMING': 'GAMFACTOR',
    'GAMES': 'GAMEOPS',
    'ULTIMATE': 'ULTGAMES',
    'T-BULL': 'TBULL',
    'ONCOARENDI': 'OAT',
    'GRIFFIN': 'GPRE',
    'BIURO': 'BIK',
    'CELON': 'CLNPHARMA',
    'AUTO PARTNER': 'AUTOPARTN',
    'X-TRADE': 'XTRADEBDM',
    'POLSKI BANK KOMÓREK MACIERZYSTYCH': 'PBKM',
    'MASTER PHARM': 'MASTERPHA',
    'KRYNICA': 'KRVITAMIN',
    'KORPORACJA': 'KGL',
    'IZO-BLOK': 'IZOBLOK',
    'PRIVATE': 'PEMANAGER',
    'BIOMED-LUBLIN': 'BIOMEDLUB',
    'BANCO': 'SANTANDER',
    'WDM CAPITAL': 'WDMCP',
    'INTERNET': 'IMS',
    'TERMO-REX': 'TERMOREX',
    'CUBE.ITG': 'CUBEITG',
    'COMPERIA.PL': 'COMPERIA',
    'MEDICALGORITHMICS': 'MEDICALG',
    'VISTAL GDYNIA': 'VISTAL',
    'MLP GROUP': 'MLPGROUP',
    'E-KANCELARIA': 'EKANCELAR',
    'PLATINUM': 'PPG',
    'TARCZYŃSKI': 'TARCZYNSKI',
    'SCO-PAK': 'SCOPAK',
    'IMMOFINANZ': 'IIAAV',
    'INTERNATIONAL': 'PROVIDENT',
    'POLSKI': 'PHN',
    'PHZ': 'BALTONA',
    'CZERWONA': 'CZTOREBKA',
    'KDM SHIPPING': 'KDMSHIPNG',
    'PCC EXOL': 'PCCEXOL',
    'MO-BRUK': 'MOBRUK',
    'SOLAR COMPANY': 'SOLAR',
    'POLSKA': 'PGODLEW',
    'JHM DEVELOPMENT': 'JHMDEV',
    'P.R.E.S.C.O.': 'PRESCO',
    'ENEL-MED': 'ENELMED',
    'BGŻ': 'BGZ',
    'INDUSTRIAL': 'IMCOMPANY',
    'EUROIMPLANT': 'EUIMPLANT',
    'E-STAR': 'ESTAR',
    'AVIA SOLUTIONS': 'AVIASG',
    'SADOVAYA GROUP': 'SADOVAYA',
    'EUROPEJSKIE': 'EUCO',
    'EKO EXPORT': 'EKOEXPORT',
    'POWSZECHNE': 'PTI',
    'GIEŁDA': 'GPW',
    'KRYNICIKI': 'KREC',
    'INTERBUD-LUBLIN': 'INTERBUD',
    'KULCZYK': 'KOV',
    'DOLNOŚLĄSKIE': 'DSS',
    'LW': 'BOGDANKA',
    'GPM': 'VINDEXUS',
    'CENTRUM': 'CENTKLIMA',
    'CHEMOSERVIS-DWORY': 'CHEMOS',
    'NWR': 'NEWWORLDR',
    'TOWARZYSTWO': 'SKOK',
    'CYFROWY': 'CYFRPLSAT',
    'ZAKŁAD': 'ZREMB',
    'BELVEDERE': 'SOBIESKI',
    'POWER': 'PWRMEDIA',
    'ZM': 'HERMAN',
    'POL-MOT': 'WARFAMA',
    'EUROPEJSKI': 'EFH',
    'UNICREDITO': 'UNICREDIT',
    'KONSORCJUM': 'KONSSTALI',
    'SECO/WARWICK': 'SECOGROUP',
    'INTEGER.PL': 'INTEGERPL',
    'ASBISC': 'ASBIS',
    'PANI': 'TERESA',
    'IZOLACJA-JAROCIN': 'IZOLACJA',
    'KOSZALIŃSKIE': 'KPPD',
    'ENERGOINSTAL': 'ENERGOINS',
    'HYDROBUDOWA': 'HBWLOCLAW',
    'KRAKCHEMIA': 'KRAKCHEM',
    'ZUK': 'STAPORKOW',
    'KRAKOWSKA': 'ARMATURA',
    'POL-AQUA': 'POLAQUA',
    'P.A.': 'PANOVA',
    'PETROLINVEST': 'PETROLINV',
    'KOMPUTRONIK': 'KOMPUTRON',
    'J.W.CONSTRUCTION': 'JWCONSTR',
    'AUTOMOTIVE': 'ACE',
    'MAKARONY': 'MAKARONPL',
    'ELEKTROTIM': 'ELEKTROTI',
    'TELFORCEONE': 'TFONE',
    'ES-SYSTEM': 'ESSYSTEM',
    'GADU-GADU': 'GADUGADU',
    'INFOVIDE-MATRIX': 'IVMX',
    'CENTRAL': 'CEDC',
    'CINEMA': 'CCIINT',
    'ŻURAWIE': 'ZURAWIE',
    'MULTIMEDIA': 'MMPPL',
    'ONE-2-ONE': 'ONE2ONE',
    'INTERFERIE': 'INTERFERI',
    'QUMAK-SEKOM': 'QUMAKSEK',
    'INTERSPORT': 'INTERSPPL',
    'EUROFILMS': 'ERGIS',
    'NORTD': 'NORTCOAST',
    'PUŁAWY': 'PULAWY',
    'TRAVELPLANET.PL': 'TRAVELPL',
    'POLISH': 'PEP',
    'ZĄBKOWICE-ERG': 'ZTSERG',
    'ŚRUBEX': 'SRUBEX'
}

In [51]:
# Test - the length should be 113
pd.merge(
    left=ipo.reset_index()[['date', 'closing_price', 'company_name', 'company_name_short']],
    right=matched.loc[(matched['company_name_short'] != matched['shares_name_0']) & (matched['shares_name_0'].isna())][['company_name_short', 'shares_name_0']],
    left_on='company_name_short',
    right_on='company_name_short',
    how='inner'
)['company_name_short'].replace(to_replace=dictionary).dropna()

0        HUUUGE
1       ANSWEAR
2       ALLEGRO
3     GAMFACTOR
4       GAMEOPS
        ...    
73       PULAWY
74     TRAVELPL
75          PEP
76       ZTSERG
77       SRUBEX
Name: company_name_short, Length: 78, dtype: object

In [52]:
# Replace the *company_name_short* with the accurate names
ipo['company_name_short'] = ipo['company_name_short'].str.replace(pat=u'\xa0', repl=' ').str.rstrip().replace(to_replace=dictionary)
ipo

,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving,closing_price,company_name_short
date,,,,,,,,,,,,
2022-07-11,CREOTECH INSTRUMENTS,100.0,39655800.0,3.965580e+07,142.00,128.8,NaN,True,False,True,128.80,CREOTECH
2021-12-20,BIO PLANET,28.0,5600000.0,9.824612e+06,26.60,27.0,4.224612e+06,True,True,True,26.83,BIO
2021-12-10,STS HOLDING,23.0,NaN,1.078125e+09,23.70,NaN,1.078125e+09,False,True,False,23.70,STSHOLDING
2021-12-09,GRUPA PRACUJ,74.0,NaN,1.119937e+09,70.00,NaN,1.119937e+09,False,True,False,70.00,GRUPA
2021-12-07,BIG CHEESE STUDIO,35.0,4725000.0,2.790424e+07,34.50,34.5,2.317924e+07,True,True,False,34.50,BIGCHEESE
...,...,...,...,...,...,...,...,...,...,...,...,...
2005-02-10,CIECH,24.0,196895616.0,2.773170e+08,28.10,28.0,8.042141e+07,True,True,False,28.03,CIECH
2005-02-04,EUROCASH,3.1,NaN,1.782001e+08,3.26,NaN,1.782001e+08,False,True,False,3.26,EUROCASH
2005-01-27,ZELMER,13.2,NaN,1.690380e+08,17.50,NaN,1.690380e+08,False,True,False,17.50,ZELMER


##### Case 1d
company_name_short != shares_name_0 & shares_name_0 != NaN & shares_name_1 != NaN

In [53]:
boolean_mask = ((matched['company_name_short'] != matched['shares_name_0']) & ~(matched['shares_name_0'].isna()) & ~(matched['shares_name_1'].isna()))
matched.loc[boolean_mask].shape

(17, 7)

In [54]:
# Create a dictionary manually using stocks dataframe
stocks[stocks['company_name'].str.startswith(pat='PC')].drop_duplicates(subset=['company_name'], keep='first').sort_values(by='company_name')

,company_name,ISIN
date,,
2012-09-20,PCCEXOL,PLPCCEX00010
2010-02-08,PCCINTER,PLPCCIM00014
2014-06-25,PCCROKITA,PLPCCRK00076
2020-12-18,PCFGROUP,PLPCFGR00010
2005-10-05,PCGUARD,PLGUARD00019
2014-04-16,PCM,PLPRMCM00048


In [55]:
dictionary = {
    'BIO': 'BIOPLANET',
    'ML': 'MLSYSTEM',
    'PLAY': 'PLAY',
    'VIGO': 'VIGOSYS',
    'ALTUS': 'ALTUSTFI',
    'OT': 'OTLOG',
    'GLOBAL': 'GLCOSMED',
    'ZE': 'ZEPAK',
    'GET': 'GETBANK',
    'AC': 'ACAUTOGAZ',
    'NOVA': 'NOVAKBM',
    'PRAGMA': 'PRAGMAINK',
    'EKO': 'EKO',
    'AD': 'DRAGOWSKI',
    'ZA': 'AZOTYTARNOW',
    'K2': 'K2INTERNT',
    'CP': 'CPENERGIA',
    'KREDYT': 'KREDYTIN',
    'PC': 'PCGUARD'
}

In [56]:
# Sense check - it should be 17 rows
pd.merge(
    left=ipo.reset_index()[['date', 'closing_price', 'company_name', 'company_name_short']],
    right=matched.loc[(matched['company_name_short'] != matched['shares_name_0']) & ~(matched['shares_name_0'].isna()) & ~(matched['shares_name_1'].isna())][['company_name_short', 'shares_name_0']],
    left_on='company_name_short',
    right_on='company_name_short',
    how='inner'
)['company_name_short'].replace(to_replace=dictionary).dropna().shape

(17,)

In [57]:
# Replace the *company_name_short* with the accurate names
ipo['company_name_short'] = ipo['company_name_short'].str.replace(pat=u'\xa0', repl=' ').str.rstrip().replace(to_replace=dictionary)
ipo

,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving,closing_price,company_name_short
date,,,,,,,,,,,,
2022-07-11,CREOTECH INSTRUMENTS,100.0,39655800.0,3.965580e+07,142.00,128.8,NaN,True,False,True,128.80,CREOTECH
2021-12-20,BIO PLANET,28.0,5600000.0,9.824612e+06,26.60,27.0,4.224612e+06,True,True,True,26.83,BIOPLANET
2021-12-10,STS HOLDING,23.0,NaN,1.078125e+09,23.70,NaN,1.078125e+09,False,True,False,23.70,STSHOLDING
2021-12-09,GRUPA PRACUJ,74.0,NaN,1.119937e+09,70.00,NaN,1.119937e+09,False,True,False,70.00,GRUPA
2021-12-07,BIG CHEESE STUDIO,35.0,4725000.0,2.790424e+07,34.50,34.5,2.317924e+07,True,True,False,34.50,BIGCHEESE
...,...,...,...,...,...,...,...,...,...,...,...,...
2005-02-10,CIECH,24.0,196895616.0,2.773170e+08,28.10,28.0,8.042141e+07,True,True,False,28.03,CIECH
2005-02-04,EUROCASH,3.1,NaN,1.782001e+08,3.26,NaN,1.782001e+08,False,True,False,3.26,EUROCASH
2005-01-27,ZELMER,13.2,NaN,1.690380e+08,17.50,NaN,1.690380e+08,False,True,False,17.50,ZELMER


#### Case 2: Companies with duplicated `company_name_short`

In [58]:
ipo[ipo.duplicated(subset=['company_name_short'], keep=False)].sort_values(by=['company_name_short'])[['company_name', 'company_name_short']].shape[0]

19

In [59]:
# Do the mapping manually

In [60]:
# Find the right names in the stocks dataframe manually
stocks[stocks['company_name'].str.startswith(pat='ZNT')].drop_duplicates(subset=['company_name'], keep='first').sort_values(by='company_name')

,company_name,ISIN
date,,
2007-11-07,ZNTKLAPY,PLZNTK000019


In [61]:
# Find the right names in the stocks dataframe manually
stocks[stocks['ISIN'].str.contains(pat='PLMRVDV00011', regex=False)].drop_duplicates(subset=['company_name'], keep='first').sort_values(by='company_name')

,company_name,ISIN
date,,
2017-12-19,MARVIPOL,PLMRVDV00011


In [62]:
ipo.loc[ipo['company_name'] == 'AB', 'company_name_short'] = 'ABPL'
ipo.loc[ipo['company_name'] == 'AB „NOVATURAS”', 'company_name_short'] = 'NOVATURAS'
ipo.loc[ipo['company_name'] == 'AB INTER RAO', 'company_name_short'] = 'INTERAOLT'

ipo.loc[ipo['company_name'] == 'ASSECO SOUTD EASTERN', 'company_name_short'] = 'ASSECOSEE'
ipo.loc[ipo['company_name'] == 'ASSECO BUSINESS SOLUTIONS', 'company_name_short'] = 'ASSECOBS'
ipo.loc[ipo['company_name'] == 'ASSECO SLOVAKIA', 'company_name_short'] = 'ASSECOSLO'

ipo.loc[ipo['company_name'] == 'CITY INTERACTIVE', 'company_name_short'] = 'CITYINTER'
ipo.loc[ipo['company_name'] == 'CITY SERVICE', 'company_name_short'] = 'CITYSERV'

ipo.loc[ipo['company_name'] == 'DOM DEVELOPMENT', 'company_name_short'] = 'DOMDEV'
ipo.loc[ipo['company_name'] == 'DOM MAKLERSKI WDM', 'company_name_short'] = 'DMWDM'

ipo.loc[ipo['company_name'] == 'GRUPA ADV', 'company_name_short'] = 'ADVGRUPA'
ipo.loc[ipo['company_name'] == 'GRUPA KOLASTYNA', 'company_name_short'] = 'KOLASTYNA'
ipo.loc[ipo['company_name'] == 'GRUPA NOKAUT', 'company_name_short'] = 'NOKAUT'
ipo.loc[ipo['company_name'] == 'GRUPA FINANSOWA PREMIUM', 'company_name_short'] = 'GFPREMIUM'
ipo.loc[ipo['company_name'] == 'GRUPA PRACUJ', 'company_name_short'] = 'GRUPRACUJ'
ipo.loc[ipo['company_name'] == 'GRUPA KOŚCIUSZKO', 'company_name_short'] = 'POLJADLO'

ipo.loc[ipo['company_name'] == 'IDEA BANK', 'company_name_short'] = 'IDEABANK'
ipo.loc[ipo['company_name'] == 'IDEA TFI', 'company_name_short'] = 'IDEATFI'

ipo.loc[ipo['company_name'] == 'MARVIPOL', 'company_name_short'] = 'MARVIPOL' #PLMRVPL00016
ipo.loc[ipo['company_name'] == 'MARVIPOL DEVELOPMENT', 'company_name_short'] = 'MARVIPOLD' #PLMRVDV00011
stocks.loc[stocks['ISIN'] == 'PLMRVDV00011', 'name'] = 'MARVIPOLD'

ipo.loc[ipo['company_name'] == 'ORZEŁ BIAŁY', 'company_name_short'] = 'ORZBIALY'
ipo.loc[ipo['company_name'] == 'ORZEŁ', 'company_name_short'] = 'ORZEL'

ipo.loc[ipo['company_name'] == 'PCC ROKITA', 'company_name_short'] = 'PCCROKITA'
ipo.loc[ipo['company_name'] == 'PCC INTERMODAL', 'company_name_short'] = 'PCCINTER'

ipo.loc[ipo['company_name'] == 'POLMOS LUBLIN', 'company_name_short'] = 'POLMOSLBN'
ipo.loc[ipo['company_name'] == 'POLMOS BIAŁYSTOK', 'company_name_short'] = 'POLMOSBIA'

ipo.loc[ipo['company_name'] == 'PZ CORMAY', 'company_name_short'] = 'CORMAY'
ipo.loc[ipo['company_name'] == 'PZU', 'company_name_short'] = 'PZU'

ipo.loc[ipo['company_name'] == 'TEN SQUARE GAMES', 'company_name_short'] = 'TSGAMES'
ipo.loc[ipo['company_name'] == 'TENDERHUT', 'company_name_short'] = 'TENDERHUT'

ipo.loc[ipo['company_name'] == 'ZAKŁADY PRZEMYSŁU CUKIERNICZEGO OTMUCHÓW', 'company_name_short'] = 'OTMUCHOW'
ipo.loc[ipo['company_name'] == 'ZAKŁADY NAPRAWCZE TABORU KOLEJOWEGO W ŁAPACH', 'company_name_short'] = 'ZNTKLAPY'

#### Final merge

In [63]:
ipo.head()

,company_name,issue_or_reference_price,new_issue_value,new_issue_and_existing_shares_value,shares_close_price,pda_close_price,existing_shares_value,is_offering_new_shares,is_offering_existing_shares,is_moving,closing_price,company_name_short
date,,,,,,,,,,,,
2022-07-11,CREOTECH INSTRUMENTS,100.0,39655800.0,3.965580e+07,142.0,128.8,NaN,True,False,True,128.80,CREOTECH
2021-12-20,BIO PLANET,28.0,5600000.0,9.824612e+06,26.6,27.0,4.224612e+06,True,True,True,26.83,BIOPLANET
2021-12-10,STS HOLDING,23.0,NaN,1.078125e+09,23.7,NaN,1.078125e+09,False,True,False,23.70,STSHOLDING
2021-12-09,GRUPA PRACUJ,74.0,NaN,1.119937e+09,70.0,NaN,1.119937e+09,False,True,False,70.00,GRUPRACUJ
2021-12-07,BIG CHEESE STUDIO,35.0,4725000.0,2.790424e+07,34.5,34.5,2.317924e+07,True,True,False,34.50,BIGCHEESE


In [64]:
# Finally, merge companies and shares dataframes
ipo = pd.merge(
    left=ipo.reset_index(),
    right=stocks.drop_duplicates(subset=['company_name'], keep='first')[['company_name', 'ISIN']].rename(columns={'company_name': 'stocks_name'}),
    left_on='company_name_short',
    right_on='stocks_name',
    how='left'
).set_index(keys='date')

print(f'''There is {ipo['ISIN'].isna().sum()} ISIN codes missing.''')

There is 0 ISIN codes missing.


### Add a new boolean column `is_privatized`

In [65]:
# Privatization consisted in selling shares in companies owned by the State Treasury

# https://www.sii.org.pl/15129/analizy/newsroom/wielcy-nieobecni-na-gpw-nie-tylko-maspex-techland-i-skarb-panstwa.html
# https://www.parkiet.com/gospodarka/art25077131-prywatyzacja-ruchu-przez-gielde
# https://www.parkiet.com/gospodarka/art25328211-polmos-bialystok-moze-trafic-na-gielde
# https://www.parkiet.com/gospodarka/art24969811-srubex-planuje-w-lutym-debiut-na-gpw
# http://www.aliorbank.pl/resources/res/repozytorium_dokumentow/biuro_maklerskie/informacje_o_debiutach.pdf
# https://biznes.interia.pl/gieldy/aktualnosci/news-debiut-bgz-pod-koniec-i-polrocza-2011-roku,nId,3727542
# https://www.bankier.pl/wiadomosc/Ta-hossa-na-GPW-jest-jakas-dziwna-7549224.html

names = [
      'CIECH', 'LOTOS', 'PGNIG', 'BOGDANKA', 'ENEA', 'PGE', 'AZOTYTARNOW', 
      'RUCH', 'SRUBEX', 'POLMOSBIA', 'PZU', 'TAURONPE', 'GPW', 'JSW', 
      'ZEPAK', 'PHN', 'PKPCARGO', 'ENERGA', 'BGZ', 'POLICE', 'PULAWY'
]
boolean_mask = ipo['company_name_short'].isin(values=names)
ipo['is_privatized'] = boolean_mask

### Add boolean column `is_foreign`

In [66]:
boolean_mask = ipo['ISIN'].str[:2] != 'PL'
ipo['is_foreign'] = boolean_mask

### Add `initial return` column

In [67]:
ipo.index.name = 'datetime'
ipo['ir'] = (ipo['closing_price'] - ipo['issue_or_reference_price']) / ipo['issue_or_reference_price']
ipo['ir'] = ipo['ir'].round(4) * 100

### Add `retail_reduction_rate`, `retail_subscription_deadline`, and `sector` columns

In [68]:
# reduction rate of subscriptions made by retail investors
retail_data = pd.read_excel(io='data/ipo_raw.xlsx', sheet_name=1)
retail_data.columns = ['date', 'company_name', 'sector', 'retail_reduction_rate', 'retail_subscription_deadline']
retail_data = retail_data.set_index(keys='company_name')
retail_data = retail_data.sort_index(ascending=False)
retail_data

,date,sector,retail_reduction_rate,retail_subscription_deadline
company_name,,,,
ŻURAWIE WIEŻOWE,2006-12-06,heavy_industry,98.55,2006-11-21
ŚRUBEX,2005-02-17,raw_materials,96.34,2005-02-09
ZĄBKOWICE-ERG,2005-02-18,raw_materials,89.93,2005-02-08
ZUK STĄPORKÓW,2007-08-03,heavy_industry,97.55,2007-07-27
ZUE,2010-10-01,heavy_industry,59.11,2010-09-16
...,...,...,...,...
AB „NOVATURAS”,2018-03-21,media,40.00,2018-03-07
AB INTER RAO,2012-12-18,NaN,NaN,NaT
AB,2006-09-21,information_technology,86.30,2006-09-06


In [69]:
ipo = ipo.join(other=retail_data[['retail_reduction_rate', 'retail_subscription_deadline', 'sector']], how='left', on='company_name')
ipo = ipo.dropna(subset=['retail_subscription_deadline'], how='any').copy()

### Add `market_volatility` and `market_return` columns

In [70]:
wig_1d = pd.read_csv(filepath_or_buffer='https://stooq.com/q/d/l/?s=wig&i=d', parse_dates=True, usecols=['Date', 'Close', 'Open'], index_col=0)
wig_1d = wig_1d.rename(columns={'Close': 'wig_c', 'Open': 'wig_o'})
wig_1d['wig_daily_return'] = ((wig_1d['wig_c'] - wig_1d['wig_o']) / wig_1d['wig_o']).round(4) * 100
wig_1d['return'] = wig_1d['wig_c'].pct_change(periods=1).round(4) * 100
wig_1d['market_volatility'] = wig_1d['return'].rolling(window=20).std().round(2)
wig_1d

,wig_o,wig_c,wig_daily_return,return,market_volatility
Date,,,,,
1991-04-16,1000.00,1000.00,0.00,NaN,NaN
1991-04-23,967.70,967.70,0.00,-3.23,NaN
1991-04-30,945.60,945.60,0.00,-2.28,NaN
1991-05-14,939.60,939.60,0.00,-0.63,NaN
1991-05-21,966.10,966.10,0.00,2.82,NaN
...,...,...,...,...,...
2023-06-02,64242.58,65654.39,2.20,3.17,1.38
2023-06-05,65882.40,65683.04,-0.30,0.04,1.37
2023-06-06,65719.15,66567.50,1.29,1.35,1.40


In [71]:
# add market_volatility on the day before the ipo (shift)
ipo = ipo.join(other=wig_1d['market_volatility'].shift(periods=1))

# add 1d return to calculate net return from IPO
ipo = ipo.join(other=wig_1d['wig_daily_return'])

In [72]:
# add wig close on the day before the ipo (shift)
ipo = ipo.join(other=wig_1d['wig_c'].shift(periods=1)).rename(columns={'wig_c': 'wig_ipo'})

# add wig close on the subscription deadline
ipo = ipo.join(other=wig_1d['wig_c'], on='retail_subscription_deadline').rename(columns={'wig_c': 'wig_deadline'})

# calculate market return based on subscription deadline
ipo = ipo.eval(expr='market_return = (wig_ipo - wig_deadline) / wig_deadline * 100')
ipo['market_return'] = ipo['market_return'].round(2)

### Add `air` and `target` columns

In [73]:
ipo['air'] = ipo['ir'] - ipo['wig_daily_return']
ipo['target'] = (ipo['air'] > 0).astype(dtype=int)

### Add `date_gap` column

In [74]:
ipo.index.name = 'date'
ipo = ipo.reset_index()
ipo['date_gap'] = ipo.apply(lambda row: len(pd.bdate_range(start=row['retail_subscription_deadline'], end=row['date'], inclusive='both')), axis=1)
ipo = ipo.set_index(keys='date')

### Add `reference_rate` column

In [75]:
# Source: https://nbp.pl/podstawowe-stopy-procentowe-archiwum/
nbp = pd.read_excel('data/reference_rate.xlsx', index_col=0, parse_dates=True)
nbp = nbp.resample(rule='1d')['stopa_referencyjna'].first().ffill()
nbp.name = 'reference_rate'
ipo = ipo.join(other=nbp)

### Add `year` column

In [76]:
ipo['year'] = ipo.index.year

## Save the final version to pickle

In [77]:
# Final check
ipo.isna().sum()

company_name                             0
issue_or_reference_price                 0
new_issue_value                         38
new_issue_and_existing_shares_value      1
shares_close_price                     136
pda_close_price                        111
existing_shares_value                  170
is_offering_new_shares                   0
is_offering_existing_shares              0
is_moving                                0
closing_price                            0
company_name_short                       0
stocks_name                              0
ISIN                                     0
is_privatized                            0
is_foreign                               0
ir                                       0
retail_reduction_rate                    0
retail_subscription_deadline             0
sector                                   0
market_volatility                        0
wig_daily_return                         0
wig_ipo                                  0
wig_deadlin

In [78]:
ipo.loc[ipo['new_issue_and_existing_shares_value'].isna(), 'new_issue_and_existing_shares_value'] = 14400000
ipo['log_value'] = np.log(ipo['new_issue_and_existing_shares_value'])
ipo['log_price'] = np.log(ipo['issue_or_reference_price'])

In [79]:
ipo['is_offering_new_shares'] = np.where(ipo['is_offering_new_shares'] == True, 1, 0)
ipo['is_offering_existing_shares'] = np.where(ipo['is_offering_existing_shares'] == True, 1, 0)
ipo['is_moving'] = np.where(ipo['is_moving'] == True, 1, 0)
ipo['is_privatized'] = np.where(ipo['is_privatized'] == True, 1, 0)
ipo['is_foreign'] = np.where(ipo['is_foreign'] == True, 1, 0)
ipo['const'] = 1

In [80]:
ipo.index.name = 'listing_date'

In [81]:
to_drop = [
    'wig_ipo', 'wig_deadline', 'company_name_short',
    'stocks_name', 'shares_close_price', 'pda_close_price',
    'retail_subscription_deadline', 'new_issue_and_existing_shares_value',
    'issue_or_reference_price', 'new_issue_value', 'existing_shares_value',
    'closing_price', 'ISIN', 'ir', 'wig_daily_return'
]
ipo = ipo.drop(columns=to_drop).copy()

In [82]:
# Drop two ipos that occured on the same day
boolean_mask = ipo.index.duplicated(keep=False)
ipo = ipo.loc[~boolean_mask].copy()

# Drop the first ipo in the research sample
ipo['target_lag1'] = ipo['target'].shift(periods=1)
ipo = ipo.dropna().copy()

In [83]:
ipo

,company_name,is_offering_new_shares,is_offering_existing_shares,is_moving,is_privatized,is_foreign,retail_reduction_rate,sector,market_volatility,market_return,air,target,date_gap,reference_rate,year,log_value,log_price,const,target_lag1
listing_date,,,,,,,,,,,,,,,,,,,
2005-01-14,COMP,1,0,0,0,0,0.00,information_technology,0.75,-2.87,14.69,1,12,6.50,2005,16.851064,3.624341,1,1.0
2005-01-27,ZELMER,0,1,0,0,0,98.70,heavy_industry,0.75,1.20,32.19,1,8,6.50,2005,18.945634,2.580217,1,1.0
2005-02-04,EUROCASH,0,1,0,0,0,43.80,retail_and_consumer_industry,0.68,1.12,3.87,1,5,6.50,2005,18.998418,1.131402,1,1.0
2005-02-10,CIECH,1,1,0,1,0,95.63,raw_materials,0.75,4.01,16.10,1,11,6.50,2005,19.440672,3.178054,1,1.0
2005-02-11,GRAAL,1,0,0,0,0,0.00,retail_and_consumer_industry,0.75,4.21,-5.93,0,11,6.50,2005,16.901820,2.708050,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-07,BIG CHEESE STUDIO,1,1,0,0,0,68.68,information_technology,1.08,-7.80,-2.20,0,21,1.25,2021,17.144289,3.555348,1,0.0
2021-12-09,GRUPA PRACUJ,0,1,0,0,0,87.67,information_technology,1.17,-0.57,-5.03,0,12,1.75,2021,20.836538,4.304065,1,0.0
2021-12-10,STS HOLDING,0,1,0,0,0,87.20,media,1.16,-0.78,2.64,1,12,1.75,2021,20.798489,3.135494,1,0.0


In [84]:
ipo.to_pickle(path='data/ipo_preprocessed.pkl')